In [1]:
import requests
import pandas as pd
import json

def EarningsCallTranscript (company, sym):

    url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/list"

    # Change company_name and symbol according to the company you are interested in
    company_name = company
    symbol = sym

    #Size controls the number of transcript details returned (not all of them are earnings call transcripts), max size is 40
    querystring = {"id":symbol ,"size":"40"}

    headers = {
	    "X-RapidAPI-Key": "f8098453cfmsh6e41a696fe434f9p10d68djsn76fb3704f8dc", #enter your key between the double quotes here
	    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    transcripts_list = response.json()

    transcripts_list_idtitle = []

    for item in transcripts_list['data']:
        transcripts_list_idtitle.append({'id': item['id'], 'Title': item['attributes']['title']})

    transcripts_list_df = pd.DataFrame(transcripts_list_idtitle)

    transcripts_useful = transcripts_list_df.loc[transcripts_list_df['Title'].str.find('Earnings Call Transcript')>=0]

    url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/get-details"

    headers = {
	    "X-RapidAPI-Key": "f8098453cfmsh6e41a696fe434f9p10d68djsn76fb3704f8dc", #enter your key between the double quotes here
	    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }

    for ind in transcripts_useful.index:
	    response = requests.get(url, headers=headers, params={"id": transcripts_useful['id'][ind]} )
	    filename = transcripts_useful['Title'][ind]
	    f = open(company_name+'/'+filename, 'w')
	    json.dump(response.json(), f)
	    f.close()

    min_date = str(transcripts_list['meta']['page']['minmaxPublishOn']['min'] - 1)
     
    #The function returns the starting point for the next iteration
    return min_date 

In [3]:
#Example: Note that you still need to create a folder with name 'Texas Instrument Incorporated' in the same directory as this scraping file
EarningsCallTranscript('Texas Instrument Incorporated', 'txn')

1575400281

In [4]:
#This function is used for repetition since we can only get 40 documents at one time
def EarningsCallTranscriptRepeat (company, sym, before):
    url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/list"

    # Change company_name and symbol according to the company you are interested in
    company_name = company
    symbol = sym

    #Size controls the number of transcript details returned (not all of them are earnings call transcripts), max size is 40
    querystring = {"id":symbol ,"size":"40", "until":before}

    headers = {
	    "X-RapidAPI-Key": "f8098453cfmsh6e41a696fe434f9p10d68djsn76fb3704f8dc", #enter your key between the double quotes here
	    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    transcripts_list = response.json()

    transcripts_list_idtitle = []

    for item in transcripts_list['data']:
        transcripts_list_idtitle.append({'id': item['id'], 'Title': item['attributes']['title']})

    transcripts_list_df = pd.DataFrame(transcripts_list_idtitle)

    transcripts_useful = transcripts_list_df.loc[transcripts_list_df['Title'].str.find('Earnings Call Transcript')>=0]
    transcripts_useful = transcripts_useful.loc[transcripts_useful['Title'].str.find(sym.upper())>=0]

    url = "https://seeking-alpha.p.rapidapi.com/transcripts/v2/get-details"

    headers = {
	    "X-RapidAPI-Key": "f8098453cfmsh6e41a696fe434f9p10d68djsn76fb3704f8dc", #enter your key between the double quotes here
	    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }

    for ind in transcripts_useful.index:
	    response = requests.get(url, headers=headers, params={"id": transcripts_useful['id'][ind]} )
	    filename = transcripts_useful['Title'][ind]
	    f = open(company_name+'/'+filename, 'w')
	    json.dump(response.json(), f)
	    f.close()

    min_date = transcripts_list['meta']['page']['minmaxPublishOn']['min'] - 1
     
    return min_date

In [5]:
#Example of repetition
EarningsCallTranscriptRepeat ('Texas Instrument Incorporated', 'txn', '1575400281')

1384917304